In [9]:
#import pdb
import numpy as np
# Coarse-grained neural network

def phi(x, a=8):
    """Funkcja aktywacji"""  
    x[x > 1] = 1
    x[x < 0] = 0
    y = np.exp(-a*(1-x)**2)      
    return y


class Settings():
    """Przechowuje ustawienia symulacji """
    def __init__(self):
        self.p_reward = [0.9, 0.15]
        self.total_trials = 30

        
class SNc():
    """Sprowadza się do pojedynczej liczby, reprezentującej ilość dopaminy przekazanej do Striatum"""
    def __init__(self):
        self.DA_TONIC = 0.5
        self.DA_DIP = 0
        self.DA_BURST = 1

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class Striatum():
    """Warstwa multineuronów, pobiera sygnały z SNC, Input, PMC"""
    def __init__(self, go_type, number_of_neurons=2):
        #---------------------------- wymiary i typ
        self.number_of_neurons = number_of_neurons    # jest równa liczbie opcji do wyboru
        self.number_of_inputs = number_of_neurons
        self.go_type = go_type
        #---------------------------- wagi połączeń z innymi warstwami
        self.w_Input = abs(np.random.randn(self.number_of_inputs,number_of_neurons)*0.1 + 0.05)
        self.w_PFC_HC = 0.4
        if go_type == 'Go':
            self.w_SNc = 1
        elif go_type=='NoGo':
            self.w_SNc = -1
        #---------------------------- wpływ neuronów dopaminergicznych warstwy SNc
        self.snc = SNc()
        #---------------------------- sygnały z innych warstw i sygnały wyjściowe
        self.signalfrom_input = 0
        self.signalfrom_SNc = np.ones(number_of_neurons).reshape(1,number_of_neurons) * self.snc.DA_TONIC
        self.signalfrom_PFC = 0
        self.outcome_minus = 0
        self.outcome_plus = 0
        #----------------------------
    
    def gen_inputs(self, inputs, input_pfc=0):
        inputs = np.array(inputs)
        self.signalfrom_input = inputs.reshape(1,inputs.size)  # wektor 1 X N
        if self.go_type == 'Go':
            self.signalfrom_PFC = input_pfc
        return self.signalfrom_input
    
    def set_dopamine(self, state='tonic', player_choice=0):
        self.signalfrom_SNc = self.snc.DA_TONIC * np.ones(self.number_of_neurons).reshape(1, self.number_of_neurons)
        if player_choice > 0:
            if state=='burst':   # reward    
                self.signalfrom_SNc[0][player_choice-1] = 1
                #pdb.set_trace()
            elif state=='dip':   # punishment
                if self.go_type=='Go':
                    self.signalfrom_SNc[0][player_choice-1] = 0
                elif self.go_type=='NoGo':
                    self.signalfrom_SNc[0][player_choice-1] = -0.5
    
    def fire_minus(self, inputs):
        self.set_dopamine(state='tonic')
        
        x = self.w_SNc*self.signalfrom_SNc + \
                np.matmul(self.signalfrom_input, self.w_Input) + \
                self.w_PFC_HC*self.signalfrom_PFC    
        x = x.reshape(1,self.number_of_neurons)
        y = phi(x)
        self.outcome_minus = y
        return y
    
    def fire_plus(self, player_choice, payment):
        if payment == 'reward':
            self.set_dopamine(state='burst', player_choice=player_choice)
        elif payment == 'punishment':
            self.set_dopamine(state='dip', player_choice=player_choice)
            
        x = self.w_SNc*self.signalfrom_SNc + \
                np.matmul(self.signalfrom_input, self.w_Input) + \
                self.w_PFC_HC*self.signalfrom_PFC
        x = x.reshape(1,self.number_of_neurons)
        y = phi(x)
        self.outcome_plus = y
        return y
    
    def learn(self, alfa=0.1, beta = 0.98):
        self.w_Input += alfa*(self.outcome_plus - self.outcome_minus)
        self.w_Input *= beta
        self.w_Input[self.w_Input < 0] = 0
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class GPe():
    """Warstwa multineuronów, pobiera sygnały ze StriatumNoGo"""
    def __init__(self, number_of_neurons=2):
        #---------------------------- wymiary i typ
        self.number_of_neurons = number_of_neurons
        #---------------------------- sygnały z innych warstw i sygnały wyjściowe
        self.signalfrom_D2 = 0
        self.outcome = 0
        #----------------------------
    
    def gen_inputs(self, inputs):
        inputs = np.array(inputs)
        self.signalfrom_D2 = inputs.reshape(1,inputs.size)  # wektor 1 X N
        return self.signalfrom_D2

    def fire(self, inputs):       
        self.outcome = 1 - inputs
        return self.outcome
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class GPi():
    """Warstwa multineuronów, pobiera sygnały ze StriatumGo i GPe"""
    def __init__(self, number_of_neurons=2):
        #---------------------------- wymiary i typ
        self.number_of_neurons = number_of_neurons
        #---------------------------- sygnały z innych warstw i sygnały wyjściowe
        self.signalfrom_D1 = 0
        self.signalfrom_GPe = 0
        self.outcome = 0
        #----------------------------
    
    def gen_inputs(self, inputs_D1, inputs_gpe):
        inputs_D1 = np.array(inputs_D1)
        self.signalfrom_D1 = inputs.reshape(1,inputs_D1.size)  # wektor 1 X N
        inputs_gpe = np.array(inputs_gpe)
        self.signalfrom_GPe = inputs.reshape(1,inputs_gpe.size)  # wektor 1 X N
        return self.signalfrom_D1, self.signalfrom_GPe

    def fire(self, inputs_D1, inputs_gpe):       
        y = 1 - 0.5*inputs_D1 - 0.5*inputs_gpe
        #y[y < 0] = 0
        self.outcome = y
        return self.outcome
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class Thalamus():
    """Warstwa multineuronów, pobiera sygnały z GPi"""
    def __init__(self, number_of_neurons=2):
        #---------------------------- wymiary i typ
        self.number_of_neurons = number_of_neurons
        #---------------------------- sygnały z innych warstw i sygnały wyjściowe
        self.signalfrom_GPi = 0
        self.outcome = 0
        #----------------------------
    
    def gen_inputs(self, inputs):
        inputs = np.array(inputs)
        self.signalfrom_GPi = inputs.reshape(1,inputs.size)  # wektor 1 X N
        return self.signalfrom_GPi

    def fire(self, inputs):       
        self.outcome = 1 - inputs
        return self.outcome
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class PFC_HC():
    """Pojedynczy multineuron, informuje StriatumGo i PMC, który to instructed stimulus"""
    def __init__(self, instructed_stimulus, number_of_neurons=2):
        I = np.zeros((1,number_of_neurons))
        I[0][instructed_stimulus-1] = 1
        self.outcome = I
        #----------------------------
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
class Pmc():
    """Warstwa multineuronów, pobiera sygnały z Thalamus, PFC_HC"""
    def __init__(self, go_type, number_of_neurons=2):
        #---------------------------- wymiary
        self.number_of_neurons = number_of_neurons    # jest równa liczbie opcji do wyboru
        self.number_of_inputs = number_of_neurons
        #---------------------------- wagi połączeń z innymi warstwami
        self.w_Input = abs(np.random.randn(self.number_of_inputs,number_of_neurons)*0.1 + 0.05)
        self.w_PFC_HC = 0.05
        #---------------------------- sygnały z innych warstw i sygnały wyjściowe
        self.signalfrom_input = 0
        self.signalfrom_thalamus = 0
        self.signalfrom_PFC = 0
        self.outcome_minus = 0
        self.outcome_plus = 0
        #----------------------------
    
    def gen_inputs(self, inputs, input_thal, input_pfc=0):
        inputs = np.array(inputs)
        self.signalfrom_input = inputs.reshape(1,inputs.size)  # wektor 1 X N
        self.signalfrom_thalamus = input_thal
        return self.signalfrom_input
       
    def fire_minus(self, inputs):
        x = 0.5*self.signalfrom_thalamus + \
                0.5*np.matmul(self.signalfrom_input, self.w_Input) + \
                self.w_PFC_HC*self.signalfrom_PFC    
        x = x.reshape(1,self.number_of_neurons)
        y = phi(x)
        self.outcome_minus = y
        return y
    
    def fire_plus(self, player_choice, payment):
        x = self.w_SNc*self.signalfrom_SNc + \
                np.matmul(self.signalfrom_input, self.w_Input) + \
                self.w_PFC_HC*self.signalfrom_PFC
        x = x.reshape(1,self.number_of_neurons)
        y = phi(x)
        self.outcome_plus = y
        return y
    
    def learn(self, alfa=0.1, beta = 0.98):
        self.w_Input += alfa*(self.outcome_plus - self.outcome_minus)
        self.w_Input *= beta
        self.w_Input[self.w_Input < 0] = 0
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#tu są nowinki

In [18]:
d1 = Striatum('Go')
pfc_hc = PFC_HC(instructed_stimulus=2)
inputs = d1.gen_inputs([1,1], pfc_hc.outcome)
d1.fire_minus(inputs)
print('signalfrom_SNc:\n',d1.signalfrom_SNc)
print('d1.outcome_minus:\n', d1.outcome_minus)
d1.fire_plus(player_choice=1,payment='punishment')
print('d1.outcome_plus:\n', d1.outcome_plus)
print('signalfrom_SNc:\n',d1.signalfrom_SNc)
print('d1.w_Input:\n', d1.w_Input)
d1.learn()
print('d1.w_Input:\n', d1.w_Input)

signalfrom_SNc:
 [[ 0.5  0.5]]
d1.outcome_minus:
 [[ 0.3711839  1.       ]]
d1.outcome_plus:
 [[ 0.00300701  1.        ]]
signalfrom_SNc:
 [[ 0.   0.5]]
d1.w_Input:
 [[ 0.00454564  0.12598029]
 [ 0.14348532  0.17833165]]
d1.w_Input:
 [[ 0.          0.12346069]
 [ 0.10453428  0.17476502]]


In [21]:
d1 = Striatum('Go')
inputs = d1.gen_inputs([1,1])
d1.fire_minus(inputs)
print('signalfrom_SNc:\n',d1.signalfrom_SNc)
print('d1.outcome_minus:\n', d1.outcome_minus)
d2 = Striatum('NoGo')
inputs = d2.gen_inputs([1,1])
d2.fire_minus(inputs)
print('signalfrom_SNc:\n',d2.signalfrom_SNc)
print('d2.outcome_minus:\n', d2.outcome_minus)
gpe = GPe()
gpe.fire(d2.outcome_minus)
print('gpe.outcome:\n',gpe.outcome)
gpi = GPi()
gpi.fire(d1.outcome_minus, gpe.outcome)
print('gpi.outcome:\n',gpi.outcome)
thal = Thalamus()
thal.fire(gpi.outcome)
print('thal.outcome:\n',thal.outcome)
pfc_hc = PFC_HC(instructed_stimulus=2)


signalfrom_SNc:
 [[ 0.5  0.5]]
d1.outcome_minus:
 [[ 0.56782412  0.55382354]]
signalfrom_SNc:
 [[ 0.5  0.5]]
d2.outcome_minus:
 [[ 0.00033546  0.00033546]]
gpe.outcome:
 [[ 0.99966454  0.99966454]]
gpi.outcome:
 [[ 0.21625567  0.22325596]]
thal.outcome:
 [[ 0.78374433  0.77674404]]


In [99]:
A = np.array([[1,2],[3,4]])
print('A=\n',A)
B = np.array([2,3]).reshape(1,2)
print('B=\n',B)

C = np.matmul(B,A)
print('C=\n',C)


A=
 [[1 2]
 [3 4]]
B=
 [[2 3]]
C=
 [[11 16]]


In [100]:
y = 2 - np.array([1,3])
y[y<0] = 0
y

array([1, 0])

In [103]:
np.zeros((1,2)).shape

(1, 2)

In [8]:
mbin = np.array([0, 1, 0]).reshape(3,1)
x = np.random.rand(3,1)
x*mbin

array([[ 0.        ],
       [ 0.38117694],
       [ 0.        ]])